# pipline for A Stage

stages:
a1. 

    1. consider using pca
    2. consider droping m_status - אולי נכין טרספורמר שזורק את זה אם מקבל 0 ולא זורק אם מקבל 1
    3. fix the encoder transformer so he will work only on X

a. make the following transformers: 

    1. encoder
    2. features_select [grid: use or drop marital status]
    3. normalizetion
    4. smote [grid: (1)% (2)k for knn]
    
b. pipline the following - with gride sarch - for rf: 

    1. encoder
    2. features_select [grid: use or drop marital status]
    3. normalizetion
    4. smote [grid: (1)% (2)k for knn]
    5. rf model [grid: (1)tree regularizetion (2)num of F per tree (3)sample per tree (4)type of sampling (5)whaight (a stage)]
    6. grid it - get a finel estimator
    
c. pipline the following - with gride sarch - for xgboost: 

    1. encoder
    2. features_select [grid: use or drop marital status]
    3. normalizetion
    4. smote [grid: (1)% (2)k for knn]
    5. xgboost model [grid: (1)whaight (a stage) (2)learning rate]
    6. grid it - get a finel estimator
    
d. cv for the two models and choose the best

e. run the best on all the data and submit

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import sklearn.preprocessing as preprocessing
import matplotlib.pyplot as plt
import seaborn as sns
!pip install imbalanced-learn
from numpy import mean
#from sklearn.model_selection import cross_val_score
#from sklearn.model_selection import RepeatedStratifiedKFold
#from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import FunctionTransformer


Defaulting to user installation because normal site-packages is not writeable


In [2]:
from cancer_func import cancer_encode
from cancer_func import cancer_features_select
from cancer_func import cancer_encoder
from cancer_func import cancer_features_selector

In [3]:
# grid search k value for SMOTE oversampling for imbalanced classification
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from sklearn.preprocessing import StandardScaler



In [4]:
df = pd.read_csv('Breast_Cancer.csv')

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       4024 non-null   int64  
 1   T Stage                   4024 non-null   float64
 2   N Stage                   4024 non-null   float64
 3   6th Stage                 4024 non-null   float64
 4   Grade                     4024 non-null   float64
 5   A Stage                   4024 non-null   float64
 6   Tumor Size                4024 non-null   int64  
 7   Estrogen Status           4024 non-null   float64
 8   Progesterone Status       4024 non-null   float64
 9   Regional Node Examined    4024 non-null   int64  
 10  Reginol Node Positive     4024 non-null   int64  
 11  Race_Black                4024 non-null   float64
 12  Race_Other                4024 non-null   float64
 13  Race_White                4024 non-null   float64
 14  Marital 

# starting the pipline

In [5]:
#from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression


In [6]:
df = cancer_encode(df)
df = cancer_features_select(df)
X = df.drop(['A Stage'], axis=1, inplace=False) #sould be without  
y = df['A Stage']

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


0       0.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
4019    0.0
4020    0.0
4021    0.0
4022    0.0
4023    0.0
Name: A Stage, Length: 4024, dtype: float64

In [10]:
encoder = FunctionTransformer(cancer_encode, validate=False)
selector = FunctionTransformer(cancer_features_select, validate=False)

over = SMOTE(sampling_strategy=0.2, k_neighbors=7)
scaler = StandardScaler() #אני מנסה פה להשתמש בסקלר בלי לעטוף חזרה את הנתונים בטבלת פנדה. אם זה לא יעבוד נעטוף בפונקציה נפרדת
logistic = LogisticRegression()#max_iter=10000) #צריך להגדיר את המודל 
#log_steps = [("encoder",encoder),("selector",selector),("scaler", scaler), ("over", over), ("logistic", logistic)]
log_steps = [("scaler", scaler), ("over", over), ("logistic", logistic)]

pipe_log = Pipeline(steps=log_steps)

param_grid_log = {
    "over__sampling_strategy": [0.1,0.2,0.3,0.5],
    "over__k_neighbors": [5,6,7,8],
    "logistic__C": [1,4],
    "logistic__class_weight":[{0: 1, 1: 1},{0: 1, 1: 2},{0: 1, 1: 3},{0: 1, 1: 10}]
}


In [47]:

search = GridSearchCV(pipe_log, param_grid_log, n_jobs=5,scoring='f1')
search.fit(X, y)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)
#לא ברור לי איך אני יודע שהוא לא עשה הערכה גם על הנתונים שהוא המציא

Best parameter (CV score=0.328):
{'logistic__C': 4, 'logistic__class_weight': {0: 1, 1: 2}, 'over__k_neighbors': 7, 'over__sampling_strategy': 0.1}


# evaluate pipeline


In [7]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve, roc_auc_score


In [29]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
scores = cross_val_score(search, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(scores))

Mean ROC AUC: 0.950


In [14]:
def report(clf, X, y):
    acc = accuracy_score(y_true=y, 
                         y_pred=clf.predict(X))
    cm = pd.DataFrame(confusion_matrix(y_true=y, 
                                       y_pred=clf.predict(X)), 
                      index=clf.classes_, 
                      columns=clf.classes_)
    rep = classification_report(y_true=y, 
                                y_pred=clf.predict(X))
    return 'accuracy: {:.3f}\n\n{}\n\n{}'.format(acc, cm, rep)

In [35]:
search.predict(X)

array([0., 0., 1., ..., 0., 0., 0.])

In [39]:
print("with defult scorer:")
print(report(search, X, y))

with defult scorer:
accuracy: 0.957

      0.0  1.0
0.0  3807  125
1.0    49   43

              precision    recall  f1-score   support

         0.0       0.99      0.97      0.98      3932
         1.0       0.26      0.47      0.33        92

    accuracy                           0.96      4024
   macro avg       0.62      0.72      0.65      4024
weighted avg       0.97      0.96      0.96      4024



In [42]:
print("with roc scorer:")
print(report(search, X, y))

with roc scorer:
accuracy: 0.934

      0.0  1.0
0.0  3686  246
1.0    20   72

              precision    recall  f1-score   support

         0.0       0.99      0.94      0.97      3932
         1.0       0.23      0.78      0.35        92

    accuracy                           0.93      4024
   macro avg       0.61      0.86      0.66      4024
weighted avg       0.98      0.93      0.95      4024



In [48]:
print("with f1 scorer:")
print(report(search, X, y))

with f1 scorer:
accuracy: 0.932

      0.0  1.0
0.0  3675  257
1.0    16   76

              precision    recall  f1-score   support

         0.0       1.00      0.93      0.96      3932
         1.0       0.23      0.83      0.36        92

    accuracy                           0.93      4024
   macro avg       0.61      0.88      0.66      4024
weighted avg       0.98      0.93      0.95      4024



# random forest

In [8]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
rf = RandomForestClassifier()
rf_steps = [("scaler", scaler), ("over", over), ("rf", rf)]
pipe_rf = Pipeline(steps=rf_steps)

param_grid_rf = {
    "over__sampling_strategy": [0.1,0.2,0.3,0.5],
    "over__k_neighbors": [5,6,7,8],
    "rf__max_depth": [3,4,5,6],
    "rf__min_samples_leaf": [4,5,6,10],
    "rf__class_weight":[{0: 1, 1: 1},{0: 1, 1: 2},{0: 1, 1: 3},{0: 1, 1: 10}]
}


In [12]:

search = GridSearchCV(pipe_rf, param_grid_rf, n_jobs=5,scoring='f1')
search.fit(X, y)
print("Best parameter (CV score=%0.3f):" % search.best_score_)
print(search.best_params_)
#לא ברור לי איך אני יודע שהוא לא עשה הערכה גם על הנתונים שהוא המציא

Best parameter (CV score=0.362):
{'over__k_neighbors': 7, 'over__sampling_strategy': 0.1, 'rf__class_weight': {0: 1, 1: 2}, 'rf__max_depth': 6, 'rf__min_samples_leaf': 6}


In [ ]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=1)
scores = cross_val_score(search, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(scores))

In [15]:
print("with f1 scorer:")
print(report(search, X, y))

with f1 scorer:
accuracy: 0.958

      0.0  1.0
0.0  3779  153
1.0    14   78

              precision    recall  f1-score   support

         0.0       1.00      0.96      0.98      3932
         1.0       0.34      0.85      0.48        92

    accuracy                           0.96      4024
   macro avg       0.67      0.90      0.73      4024
weighted avg       0.98      0.96      0.97      4024



In [17]:
search.best_estimator_.feature_importances_

AttributeError: 'Pipeline' object has no attribute 'feature_importances_'

In [26]:



search.best_estimator_.named_steps['rf'].feature_importances_

array([1.76844725e-02, 1.52443199e-01, 1.89782140e-01, 3.40430639e-01,
       3.12934422e-02, 3.39234597e-02, 7.26508650e-03, 7.65134668e-03,
       3.07403444e-02, 1.07254353e-01, 1.97209676e-03, 4.64359260e-04,
       3.60232389e-03, 3.47023180e-03, 4.22237627e-03, 1.51509647e-04,
       1.78022010e-03, 1.93057448e-03, 6.39378242e-02])

In [29]:
hd = list(X.columns)
print("feature importances: ")
for i, f in zip(hd, search.best_estimator_.named_steps['rf'].feature_importances_):
     print(i,round(f*100,2))

feature importances: 
Age 1.77
T Stage  15.24
N Stage 18.98
6th Stage 34.04
Grade 3.13
Tumor Size 3.39
Estrogen Status 0.73
Progesterone Status 0.77
Regional Node Examined 3.07
Reginol Node Positive 10.73
Race_Black 0.2
Race_Other 0.05
Race_White 0.36
Marital Status_Divorced 0.35
Marital Status_Married 0.42
Marital Status_Separated 0.02
Marital Status_Single  0.18
Marital Status_Widowed 0.19
Regional_Node_pos_% 6.39


playing with the code

In [7]:
df = cancer_encode(df)
df = cancer_features_select(df)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 21 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       4024 non-null   int64  
 1   T Stage                   4024 non-null   float64
 2   N Stage                   4024 non-null   float64
 3   6th Stage                 4024 non-null   float64
 4   differentiate             4024 non-null   object 
 5   Grade                     4024 non-null   float64
 6   A Stage                   4024 non-null   float64
 7   Tumor Size                4024 non-null   int64  
 8   Estrogen Status           4024 non-null   float64
 9   Progesterone Status       4024 non-null   float64
 10  Regional Node Examined    4024 non-null   int64  
 11  Reginol Node Positive     4024 non-null   int64  
 12  Race_Black                4024 non-null   float64
 13  Race_Other                4024 non-null   float64
 14  Race_Whi

In [69]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4024 entries, 0 to 4023
Data columns (total 19 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Age                       4024 non-null   int64  
 1   T Stage                   4024 non-null   float64
 2   N Stage                   4024 non-null   float64
 3   6th Stage                 4024 non-null   float64
 4   Grade                     4024 non-null   float64
 5   Tumor Size                4024 non-null   int64  
 6   Estrogen Status           4024 non-null   float64
 7   Progesterone Status       4024 non-null   float64
 8   Regional Node Examined    4024 non-null   int64  
 9   Reginol Node Positive     4024 non-null   int64  
 10  Race_Black                4024 non-null   float64
 11  Race_Other                4024 non-null   float64
 12  Race_White                4024 non-null   float64
 13  Marital Status_Divorced   4024 non-null   float64
 14  Marital 

In [70]:
X

,Age,T Stage,N Stage,6th Stage,Grade,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Race_Black,Race_Other,Race_White,Marital Status_Divorced,Marital Status_Married,Marital Status_Separated,Marital Status_Single,Marital Status_Widowed,Regional_Node_pos_%
0,68,0.0,0.0,0.0,3.0,4,1.0,1.0,24,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,4.166667
1,50,1.0,1.0,2.0,2.0,35,1.0,1.0,14,5,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,35.714286
2,58,2.0,2.0,4.0,2.0,63,1.0,1.0,14,7,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,50.000000
3,58,0.0,0.0,0.0,3.0,18,1.0,1.0,2,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,50.000000
4,47,1.0,0.0,1.0,3.0,41,1.0,1.0,3,1,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,33.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4019,62,0.0,0.0,0.0,2.0,9,1.0,1.0,1,1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,100.000000
4020,56,1.0,1.0,2.0,2.0,46,1.0,1.0,14,8,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,57.142857
4021,68,1.0,0.0,1.0,2.0,22,1.0,0.0,11,3,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,27.272727
4022,58,1.0,0.0,1.0,2.0,44,1.0,1.0,11,1,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,9.090909


In [71]:
ssr = StandardScaler()
X_columns =X.columns
X = ssr.fit_transform(X)
X = pd.DataFrame(X, columns=X_columns)


In [72]:
X

,Age,T Stage,N Stage,6th Stage,Grade,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Race_Black,Race_Other,Race_White,Marital Status_Divorced,Marital Status_Married,Marital Status_Separated,Marital Status_Single,Marital Status_Widowed,Regional_Node_pos_%
0,1.565253,-1.025287,-0.632209,-1.043706,1.331031,-1.253661,0.267652,0.458107,1.190676,-0.618172,-0.279201,-0.293927,0.423109,-0.370629,0.722850,-0.106346,-0.424741,-0.249041,-0.992386
1,-0.443222,0.281158,0.809974,0.535491,-0.235987,0.214345,0.267652,0.458107,-0.044095,0.164807,-0.279201,-0.293927,0.423109,-0.370629,0.722850,-0.106346,-0.424741,-0.249041,0.106887
2,0.449434,1.587604,2.252157,2.114687,-0.235987,1.540287,0.267652,0.458107,-0.044095,0.556296,-0.279201,-0.293927,0.423109,2.698117,-1.383413,-0.106346,-0.424741,-0.249041,0.604671
3,0.449434,-1.025287,-0.632209,-1.043706,1.331031,-0.590691,0.267652,0.458107,-1.525820,-0.618172,-0.279201,-0.293927,0.423109,-0.370629,0.722850,-0.106346,-0.424741,-0.249041,0.604671
4,-0.777968,0.281158,-0.632209,-0.254108,1.331031,0.498475,0.267652,0.458107,-1.402343,-0.618172,-0.279201,-0.293927,0.423109,-0.370629,0.722850,-0.106346,-0.424741,-0.249041,0.023923
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4019,0.895761,-1.025287,-0.632209,-1.043706,-0.235987,-1.016886,0.267652,0.458107,-1.649297,-0.618172,-0.279201,3.402205,-2.363456,-0.370629,0.722850,-0.106346,-0.424741,-0.249041,2.346916
4020,0.226270,0.281158,0.809974,0.535491,-0.235987,0.735251,0.267652,0.458107,-0.044095,0.752041,-0.279201,-0.293927,0.423109,2.698117,-1.383413,-0.106346,-0.424741,-0.249041,0.853563
4021,1.565253,0.281158,-0.632209,-0.254108,-0.235987,-0.401271,0.267652,-2.182898,-0.414526,-0.226682,-0.279201,-0.293927,0.423109,-0.370629,0.722850,-0.106346,-0.424741,-0.249041,-0.187258
4022,0.449434,0.281158,-0.632209,-0.254108,-0.235987,0.640541,0.267652,0.458107,-0.414526,-0.618172,3.581645,-0.293927,-2.363456,2.698117,-1.383413,-0.106346,-0.424741,-0.249041,-0.820801


In [73]:

over = SMOTE(sampling_strategy=0.2, k_neighbors=7)
X_o, y_o = over.fit_resample(X, y)

df_o = pd.concat([X_o, y_o], axis=1)


In [74]:
df.corr()['A Stage']

Age                        -0.020860
T Stage                     0.221124
N Stage                     0.260573
6th Stage                   0.291962
Grade                       0.039459
A Stage                     1.000000
Tumor Size                  0.123883
Estrogen Status            -0.065570
Progesterone Status        -0.026530
Regional Node Examined      0.069010
Reginol Node Positive       0.232849
Race_Black                  0.008646
Race_Other                 -0.001943
Race_White                 -0.004776
Marital Status_Divorced     0.019841
Marital Status_Married     -0.022505
Marital Status_Separated    0.031167
Marital Status_Single       0.013582
Marital Status_Widowed     -0.016823
Regional_Node_pos_%         0.171993
Name: A Stage, dtype: float64

In [75]:
df_o.corr()['A Stage']

Age                        -0.054824
T Stage                     0.472844
N Stage                     0.570987
6th Stage                   0.634756
Grade                       0.090747
Tumor Size                  0.300400
Estrogen Status            -0.120299
Progesterone Status        -0.020517
Regional Node Examined      0.180693
Reginol Node Positive       0.497319
Race_Black                  0.005460
Race_Other                 -0.023378
Race_White                  0.013551
Marital Status_Divorced     0.026349
Marital Status_Married     -0.030517
Marital Status_Separated    0.045272
Marital Status_Single       0.040564
Marital Status_Widowed     -0.062571
Regional_Node_pos_%         0.407736
A Stage                     1.000000
Name: A Stage, dtype: float64

In [76]:
df.groupby(['A Stage']).mean()


,Age,T Stage,N Stage,6th Stage,Grade,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Race_Black,Race_Other,Race_White,Marital Status_Divorced,Marital Status_Married,Marital Status_Separated,Marital Status_Single,Marital Status_Widowed,Regional_Node_pos_%
A Stage,,,,,,,,,,,,,,,,,,,
0.0,54.000763,0.758901,0.410732,1.265259,2.146745,30.073499,0.935656,0.828077,14.271617,3.976094,0.071974,0.079603,0.848423,0.119786,0.658444,0.010682,0.152085,0.059003,31.891751
1.0,52.750000,1.891304,1.619565,3.739130,2.315217,47.576087,0.826087,0.760870,18.010870,11.934783,0.086957,0.076087,0.836957,0.163043,0.586957,0.032609,0.184783,0.032609,64.915701


In [77]:
df_o.groupby(['A Stage']).mean()


,Age,T Stage,N Stage,6th Stage,Grade,Tumor Size,Estrogen Status,Progesterone Status,Regional Node Examined,Reginol Node Positive,Race_Black,Race_Other,Race_White,Marital Status_Divorced,Marital Status_Married,Marital Status_Separated,Marital Status_Single,Marital Status_Widowed,Regional_Node_pos_%
A Stage,,,,,,,,,,,,,,,,,,,
0.0,0.003191,-0.033824,-0.039858,-0.044659,-0.006036,-0.018950,0.010030,0.004058,-0.010556,-0.035617,-0.001323,0.000297,0.000731,-0.003035,0.003442,-0.004767,-0.002078,0.002573,-0.026309
1.0,-0.142544,1.445205,1.742712,1.920454,0.233377,0.843077,-0.335101,-0.050650,0.465220,1.539520,0.013162,-0.060941,0.036445,0.067793,-0.078162,0.123017,0.107965,-0.156789,1.140693


הממוצעים יחסית נשמרים. הקורלציה מתחזקת, בעיקר בגלל שריכוז ה1 עולה ולכן יכול להיות יותר קשר בין המשתנים. לאחר נרמול אני חושב שהקורלציה לא אמורה להשתנות.